In [ ]:
#@title Подпись
#@markdown Данный блокнот предназначен для тестовой подписи .pkpass. \
#@markdown Принимает на вход экспортированные архивы с сайта https://pkvd.app/creator \
#@markdown Исключительно для тестов. Можно посмотреть в [Pass2U](https://play.google.com/store/apps/details?id=com.passesalliance.wallet&hl=en_US)

!pip3 install applepassgenerator
import json
import hashlib
from io import BytesIO
from zipfile import ZipFile

from pathlib import Path
from cryptography import x509
from cryptography.hazmat.primitives import hashes, serialization
from cryptography.hazmat.primitives.serialization import pkcs7
import requests

certificate = requests.get("https://web.archive.org/web/20231011093141/https://raw.githubusercontent.com/devartis/passbook/master/passbook/test/certificates/certificate.pem").content
key = requests.get("https://web.archive.org/web/20231011093338/https://raw.githubusercontent.com/devartis/passbook/master/passbook/test/certificates/private.key").content
wwdr = requests.get("https://web.archive.org/web/20231011093425/https://raw.githubusercontent.com/devartis/passbook/master/passbook/test/certificates/wwdr_certificate.pem").content
certs_passwd = "testing-123-drop-mic"

def sign(data:bytes, name:str):
  data_buf = BytesIO(data)
  print(len(data_buf.getvalue()))
  zip_file = ZipFile(data_buf)
  fd = Path(f"signed_{name.replace(' ', '_')}.pkpass")
  if fd.exists():
    fd.unlink()
  zf = ZipFile(fd, "w")

  hashesd = {}
  for f in zip_file.filelist:
    if f.is_dir() or '/' in f.filename or f.filename == "manifest.json" or f.filename == "signature":
      continue
    file_data = zip_file.open(f, "r").read()
    try:
      file_data=file_data.decode().encode("utf-8")
    except:
      file_data = zip_file.open(f, "r").read()

    hashesd[f.filename] = hashlib.sha1(file_data).hexdigest()
    zf.writestr(f, file_data)
  manifest = json.dumps(hashesd)

  cert = x509.load_pem_x509_certificate(certificate.decode().encode("utf-8"))
  priv_key = serialization.load_pem_private_key(
              key.decode().encode("UTF-8"), password=certs_passwd.encode("UTF-8")
          )
  wwdr_cert = x509.load_pem_x509_certificate(
              wwdr.decode().encode("utf-8")
          )

  options = [pkcs7.PKCS7Options.DetachedSignature]
  sig = (
              pkcs7.PKCS7SignatureBuilder()
              .set_data(manifest.encode("UTF-8"))
              .add_signer(cert, priv_key, hashes.SHA1())
              .add_certificate(wwdr_cert)
              .sign(serialization.Encoding.DER, options)
          )

  zf.writestr("manifest.json", manifest)
  zf.writestr("signature", sig)
  zf.close()
  zip_file.close()

  if fd.exists():
    files.download(fd)


from google.colab import files

files_data = files.upload()
for zip_name in files_data:
    sign(files_data[zip_name], zip_name)
